In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import re
import sqlite3
from pprint import pprint
from tqdm import tqdm  
from time import sleep

#https://habr.com/ru/company/pixonic/blog/662812/
#https://habr.com//ru/company/pixonic/blog/662812/

URL_left = 'https://habr.com/ru'
URL_right = ['/flows/design/', '/flows/management/', '/flows/marketing/','/flows/develop/']
HEADERS = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36', 'accept': '*/*'}
HOST = 'https://habr.com'
FILE = 'habr.csv'

def get_html(url, params = None):
    r = requests.get(url, headers = HEADERS, params = params, timeout=5)
    return r

#собираем контент со страницы
def get_content(html):
    soup = BeautifulSoup(html, 'html.parser')
    items = soup.find_all("article", class_ = "tm-articles-list__item")
    print("Started")
    texts = []
    links = []
    for item in items:
        try:
            link = HOST + item.find('a', class_ = 'tm-article-snippet__title-link').get('href')
        except Exception:
            print('Product link missing')
            link = ''
        text = get_more_inf(link)
        texts.append({
            'link':link,
            'text':text
        })

    return texts
#собираем articles
def get_more_inf(link):
    if link == None or link == '':
        return '', '', '', ''
    text=[]
    try:
        h = get_html(link)
        soup = BeautifulSoup(h.text, 'html.parser')

        reviews = soup.find_all('p')
        for review in reviews:
            #print(review, '\n------')
            new_r = ''.join(review.findAll(text=True))
            #new_r=review.get_text(strip=True)
            text.append(new_r)
        #print(text)
        #pages = int(soup.find_all('a', class_='pageNum')[-1].get('data-page-number'))
        return text
    except Exception:
        pass

def save_file(items, path):
    with open (path, 'w', newline = '') as f:
        writer = csv.writer(f, delimiter = ';')
        writer.writerow(['link', 'text'])
        for item in items:
            writer.writerow([item['link'], item['text']])

#parse some pages
def go_through_pages(URL_left, URL_Right, ix, batches, cafes):
    for page in tqdm(range(ix, batches+ix)):
        print(ix, batches, '\n---------')
        #if batches+ix > 100:
        #   return cafes
        URL_new = URL_left+URL_right[0]+f'page{page}'
        print('Парсинг страницы', page, 'из', batches+ix)
        
        print(URL_new)
        sleep(2)
        html = get_html(URL_new)
        cafes.extend(get_content(html.text))
    return cafes
def parse():
    html = get_html(URL_left+URL_right[0])
    print(html)
    if html.status_code == 200:
        cafes = []
        #cafes.extend(get_content(html.text))
        batches = 5
        ix = 1
        for i in tqdm(range(2)):
            go_through_pages(URL_left, URL_right, ix, batches, cafes)
            print(ix, batches+ix)
            ix = batches+ix
            sleep(3)
            print(ix, batches+ix)
        return cafes
    else:
        print ('Error')

p = parse()

<Response [200]>


  0%|          | 0/5 [00:00<?, ?it/s]

1 5 
---------
Парсинг страницы 1 из 6
https://habr.com/ru/flows/design/page1
Started



 20%|██        | 1/5 [00:19<01:17, 19.42s/it]

1 5 
---------
Парсинг страницы 2 из 6
https://habr.com/ru/flows/design/page2
Started


...

6 5 
---------
Парсинг страницы 10 из 11
https://habr.com/ru/flows/design/page10
Started



100%|██████████| 5/5 [01:41<00:00, 20.27s/it]


6 11


100%|██████████| 2/2 [03:28<00:00, 104.22s/it]

11 16


In [ ]:
print(p[57])

{'link': 'https://habr.com/ru/company/koshelek/blog/663510/', 'text': ['Как часто вы задумывались, как пользователи воспринимают ваше приложение? Со стороны разработчика всё просто: вы знаете, как что работает, в каком разделе можно всё найти или настроить и, конечно же, как надо пользоваться фичами. Мы все прекрасно понимаем, в каких случаях приложение будет полезно, чем оно отличается от аналогов и почему именно оно прекрасно. Мы любим свои приложения.', 'Но если зайти в аналитику использования или поговорить с пользователями по душам, то быстро становится понятно, что не все обладают вашим знанием о приложении.\xa0', 'Та супер-мега полезная функциональность, над которой вы трудились несколько месяцев, вкладывая туда свою душу, силы и деньги, используется лишь малым количеством людей. А невероятный, продуманный, удобный и простой UX-дизайн пользователи не видят в упор. Более того, Полезная Функциональность воспринимается многими, как бесполезный гиммик, который отвлекает от центральн

In [ ]:
print(len(p))

200


In [ ]:
%cd ~/.jupyter

/root/.jupyter


In [ ]:
! ls

jupyter_notebook_config.py  migrated


In [ ]:
!cat jupyter_notebook_config.py

In [ ]:
!gedit jupyter_notebook_config.py

/bin/bash: gedit: command not found


In [ ]:
def check_eng(dictt):
    #sleep(3)
    texts_with_eng = []
    try:
        for elem in p:
            for key, value in elem.items():
                if key == 'text':
                    new_v = ' '.join(value)
                    words = new_v.split()
                    if len(words) < 20:
                        continue
                    count = 0
                    for elem2 in words:
                        if re.search(r'[a-zA-Z]', elem2):
                            count +=1
                    if count*100/len(words) > 15 and count*100/len(words) <90:
                        texts_with_eng.append(' '.join(words))
        return texts_with_eng
    except TypeError:
        print(Exception)


In [ ]:
with open('out.txt', 'w') as f:
    for elem in p:
        for key,value in elem.items():
            if key == 'text':
                #print(value)
                f.write(' '.join(value))
                f.write('\n')

In [ ]:
new_p = check_eng(p)

In [ ]:
new_p

['Если в 2018 году люди тратили на просмотр видео 1,5 часа в день, то в 2022 году вовлечённость выросла до 2,5 часов. Пользователи делятся видео в два раза чаще, чем любым другим контентом, а motion-графика почти так же популярна, как простые записи. Сделали для вас подборку обучающих ресурсов, программ и инструментов для самостоятельного обучения. В декабре 2021 года английская анимационная студия Wyzowl провела исследование, согласно которому motion-графика занимает второе место по популярности среди всех разновидностей видеоконтента (33%), уступая лишь 9% формату обычной видеозаписи (42%). Видео помогает компаниям привлекать новых клиентов, увеличивать объёмы продаж, повышать узнаваемость бренда, сокращать количество обращений в службу поддержки. Это подтверждают и сами потребители: 88% говорят, что именно просмотр видео убедил их совершить покупку, 72% предпочитают узнавать о новых продуктах и услугах из видеороликов. Рассказываем, как сделать motion-дизайн профессией. Требования р

In [ ]:
new_p

['Если в 2018 году люди тратили на просмотр видео 1,5 часа в день, то в 2022 году вовлечённость выросла до 2,5 часов. Пользователи делятся видео в два раза чаще, чем любым другим контентом, а motion-графика почти так же популярна, как простые записи. Сделали для вас подборку обучающих ресурсов, программ и инструментов для самостоятельного обучения. В декабре 2021 года английская анимационная студия Wyzowl провела исследование, согласно которому motion-графика занимает второе место по популярности среди всех разновидностей видеоконтента (33%), уступая лишь 9% формату обычной видеозаписи (42%). Видео помогает компаниям привлекать новых клиентов, увеличивать объёмы продаж, повышать узнаваемость бренда, сокращать количество обращений в службу поддержки. Это подтверждают и сами потребители: 88% говорят, что именно просмотр видео убедил их совершить покупку, 72% предпочитают узнавать о новых продуктах и услугах из видеороликов. Рассказываем, как сделать motion-дизайн профессией. Требования р

In [ ]:
new_li = []
for article in new_p:
    res = ''
    for word in article.split():
        res += re.sub(r'([?!.])$',r' \1 ',word) + ' '
    new_li.append(res)

In [ ]:
new_li

['Если в 2018 году люди тратили на просмотр видео 1,5 часа в день, то в 2022 году вовлечённость выросла до 2,5 часов .  Пользователи делятся видео в два раза чаще, чем любым другим контентом, а motion-графика почти так же популярна, как простые записи .  Сделали для вас подборку обучающих ресурсов, программ и инструментов для самостоятельного обучения .  В декабре 2021 года английская анимационная студия Wyzowl провела исследование, согласно которому motion-графика занимает второе место по популярности среди всех разновидностей видеоконтента (33%), уступая лишь 9% формату обычной видеозаписи (42%) .  Видео помогает компаниям привлекать новых клиентов, увеличивать объёмы продаж, повышать узнаваемость бренда, сокращать количество обращений в службу поддержки .  Это подтверждают и сами потребители: 88% говорят, что именно просмотр видео убедил их совершить покупку, 72% предпочитают узнавать о новых продуктах и услугах из видеороликов .  Рассказываем, как сделать motion-дизайн профессией .

In [ ]:
new_li=[]
import re
for elem in new_p:
    ne_elem = elem.split()
    #print(ne_elem)
    for word in ne_elem:
        #print(word)
        #if re.search(r'\.$', word):
            #word1 = re.sub(r'([?!.])$',r' \1 ',word)
        elem_new = word.replace(word + r'\.$', word+' . ')
        
        print(elem_new)
        new_li.append(elem_new)
#new_li

In [ ]:
elem

'За последнее время значимость голосовых функций и звука заметно выросла. Примером тому может служить уже громкая история запуска приложения Clubhouse, голосовых ассистентов Сбера и общего оживления интереса со стороны пользователей, компаний и инвесторов к звуку на мобильных устройствах. В этой статье я бы хотел рассмотреть пример разработки голосового помощника на платформе iOS, используя язык Swift. На мой взгляд, звук как интерфейс между пользователем и приложениями изначально был недооценен, однако с появлением технологий искусственного интеллекта, высокой информационной нагрузки и нехватки времени, польза аудио становится очевиднее. Например, в этой статье представлены интересные факты об использовании голоса за последние 4 года и вывод: The key, however, is the device letting the human think and speak like a human. Once we get there, this whole voice thing will become the predominant mode for input. We’re likely five to ten years away from getting there. However, many businesses

In [ ]:
with open('file_habr.txt', 'w') as f:
    for elem in new_li:
        f.write(elem)
        f.write('\n')